<a href="https://colab.research.google.com/github/fwangliberty/AIoTDesign-Frontend/blob/master/Classification_CNN1D_73_order_features_split_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN1N for CICIDS 2017 Data Set with ordered 73 Features (1) and SMOTE

We use the pre-processing dataset from mlp4nids (Multi-layer perceptron for network intrusion detection) https://github.com/ArnaudRosay/mlp4nids. The train, test and validation sets have "src host count" and "dst host count" new features. We try to use SMOTE to overcome the imbalanced dataset.

In [100]:
import os
from os.path import join
import glob
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [101]:
def display_metrics(y_test, y_pred, label_names):
  print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))

  print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
  print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
  print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

  print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
  print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
  print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

  print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
  print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
  print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

  print('\nClassification Report\n')
  print(classification_report(y_test, y_pred, target_names=label_names))

In [102]:
def display_all(df):
    with pd.option_context("display.max_rows", 100, "display.max_columns", 100): 
        print(df)

In [103]:
def make_value2index(attacks):
    #make dictionary
    attacks = sorted(attacks)
    d = {}
    counter=0
    for attack in attacks:
        d[attack] = counter
        counter+=1
    return d

In [104]:
# chganges label from string to integer/index
def encode_label(Y_str):
    labels_d = make_value2index(np.unique(Y_str))
    Y = [labels_d[y_str] for y_str  in Y_str]
    Y = np.array(Y)
    return np.array(Y)

## Step 1. Loading csv files

In [6]:
# All columns
col_names = ['dst host count', 'src host count', 'Source Port', 'Destination Port',
                      'Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets',
                      'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean',
                      'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std',
                      'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total',
                      'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
                      'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Header Length', 'Bwd Header Length',
                      'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
                      'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
                      'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd Segment Size',
                      'Avg Bwd Segment Size','Subflow Fwd Packets', 'Subflow Fwd Bytes',
                      'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward',
                      'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
                      'Idle Std', 'Idle Max', 'Idle Min', 'Label']

In [105]:
col_important = np.array(['dst host count', 'src host count', 'Source Port', 'Destination Port', 'Fwd IAT Min', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'Flow IAT Min',
                      'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets',
                      'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Protocol', 'Fwd Packet Length Min', 'Fwd Packet Length Mean',
                      'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std',
                      'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Fwd IAT Total',
                      'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
                      'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Header Length', 'Bwd Header Length',
                      'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
                      'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
                      'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd Segment Size',
                      'Avg Bwd Segment Size','Subflow Fwd Packets', 'Subflow Fwd Bytes',
                      'Subflow Bwd Packets', 'Subflow Bwd Bytes', 
                      'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
                      'Idle Std', 'Idle Max', 'Idle Min', 'Label'])

### Option 1. Connect to Google Drive

In [106]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [107]:
# load three csv files generated by mlp4nids (Multi-layer perceptron for network intrusion detection )
# first load the train set
df_train = pd.read_csv('/content/drive/My Drive/CICIDS2017/train_set_ext.csv',names=col_important, skiprows=1)  

In [108]:
print('Train set size: ', df_train.shape)

Train set size:  (556548, 74)


In [109]:
df_test = pd.read_csv('/content/drive/My Drive/CICIDS2017/test_set_ext.csv',names=col_important, skiprows=1)  
print('Test set size: ', df_test.shape)

df_val = pd.read_csv('/content/drive/My Drive/CICIDS2017/crossval_set_ext.csv',names=col_important, skiprows=1)  
print('Validation set size: ', df_val.shape)

Test set size:  (278271, 74)
Validation set size:  (278271, 74)


### Option 2. Load from local machine

In [11]:
dataroot = '../data/cicids2017clean/train_set_ext.csv'
df_train = pd.read_csv(dataroot, names=col_names, skiprows=1) 

FileNotFoundError: ignored

In [ ]:
dataroot = '../data/cicids2017clean/crossval_set_ext.csv'
df_val = pd.read_csv(dataroot, names=col_names, skiprows=1) 
dataroot = '../data/cicids2017clean/test_set_ext.csv'
df_test = pd.read_csv(dataroot, names=col_names, skiprows=1) 

## Step 2. Exploring the dataset

In [110]:
df_train.head()

,dst host count,src host count,Source Port,Destination Port,Fwd IAT Min,Init_Win_bytes_forward,Init_Win_bytes_backward,Flow IAT Min,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Protocol,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
97267,100,100,49802,80,6,81106330,8,4,56,11601,20,0,7.0,5.656854,8675,0,2900.25,4088.041942,1.437249e+02,1.479539e-01,7.373303e+06,2.180000e+07,72500000,1,80700000,11500000.0,27100000.0,72500000,1,423439,141146.3333,234485.28,411829,191,0,0,172,92,9.863595e-02,0.049318,0,8675,897.153846,2471.828063,6109933.974,0,0,0,0,1,0,0,0,0,971.916667,7.0,2900.25,8,56,4,11601,256,229,6,20,1002.0,0.0,1002,1002,40300000.0,45500000.0,72500000,8137801,DDoS
248498,19,19,57027,53,17,139,2,2,76,108,38,38,38.0,0.000000,54,54,54.00,0.000000,1.323741e+06,2.877698e+04,4.633333e+01,7.505553e+01,133,3,3,3.0,0.0,3,3,3,3.0000,0.00,3,3,0,0,40,40,1.438849e+04,14388.489210,38,54,44.400000,8.763561,76.800,0,0,0,0,0,0,0,0,1,55.500000,38.0,54.00,2,76,2,108,-1,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2311978,94,94,58114,80,6,1,2,0,0,0,0,0,0.0,0.000000,0,0,0.00,0.000000,0.000000e+00,2.000000e+06,1.000000e+00,0.000000e+00,1,1,1,1.0,0.0,1,1,0,0.0000,0.00,0,0,0,0,64,0,2.000000e+06,0.000000,0,0,0.000000,0.000000,0.000,0,0,0,0,1,0,0,0,0,0.000000,0.0,0.00,2,0,0,0,251,-1,0,32,0.0,0.0,0,0,0.0,0.0,0,0,DoS Hulk
556086,6,6,443,2123,6,34,1,1,6,6,6,6,6.0,0.000000,6,6,6.00,0.000000,3.529412e+05,5.882353e+04,3.400000e+01,0.000000e+00,34,34,0,0.0,0.0,0,0,0,0.0000,0.00,0,0,0,0,20,20,2.941176e+04,29411.764710,6,6,6.000000,0.000000,0.000,0,0,0,0,1,1,0,0,1,9.000000,6.0,6.00,1,6,1,6,119,16355,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2310971,90,90,54422,80,6,1,2,0,0,0,0,0,0.0,0.000000,0,0,0.00,0.000000,0.000000e+00,2.000000e+06,1.000000e+00,0.000000e+00,1,1,1,1.0,0.0,1,1,0,0.0000,0.00,0,0,0,0,64,0,2.000000e+06,0.000000,0,0,0.000000,0.000000,0.000,0,0,0,0,1,0,0,0,0,0.000000,0.0,0.00,2,0,0,0,251,-1,0,32,0.0,0.0,0,0,0.0,0.0,0,0,DoS Hulk


Count the number of attacks

In [ ]:
df_train['Label'].value_counts()

BENIGN                        278274
DoS Hulk                      115062
PortScan                       79402
DDoS                           64012
DoS GoldenEye                   5146
FTP-Patator                     3967
SSH-Patator                     2948
DoS slowloris                   2898
DoS Slowhttptest                2749
Bot                              978
Web Attack � Brute Force         753
Web Attack � XSS                 326
Infiltration                      18
Web Attack � Sql Injection        10
Heartbleed                         5
Name: Label, dtype: int64

In [ ]:
df_train.describe()

,dst host count,src host count,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,556548.000000,556548.000000,556548.000000,556548.000000,556548.000000,5.565480e+05,556548.000000,556548.000000,5.565480e+05,5.565480e+05,...,556548.000000,556548.000000,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05,5.565480e+05
mean,63.986114,71.911368,41059.562686,7182.692884,8.358620,2.159931e+07,7.561386,8.259167,6.822941e+02,1.265955e+04,...,4.883591,25.540766,1.082361e+05,3.242660e+04,1.571849e+05,8.865328e+04,1.640646e+07,1.206008e+06,1.730500e+07,1.552621e+07
std,40.704131,36.408934,20762.257722,16999.053361,4.518634,3.824665e+07,641.917381,865.356666,8.207379e+03,1.934124e+06,...,612.739185,6.418384,7.164700e+05,3.567006e+05,9.588739e+05,6.668469e+05,3.283531e+07,7.260774e+06,3.390042e+07,3.259279e+07
min,1.000000,1.000000,0.000000,0.000000,0.000000,-1.300000e+01,1.000000,0.000000,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,19.000000,42.000000,33830.000000,80.000000,6.000000,1.120000e+02,1.000000,1.000000,6.000000e+00,6.000000e+00,...,0.000000,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,96.000000,97.000000,49512.000000,80.000000,6.000000,6.867450e+04,2.000000,2.000000,5.600000e+01,1.120000e+02,...,1.000000,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,100.000000,100.000000,57000.000000,1034.000000,6.000000,1.104332e+07,6.000000,6.000000,3.380000e+02,1.159500e+04,...,2.000000,32.000000,9.530000e+02,0.000000e+00,9.530000e+02,8.820000e+02,9.553880e+06,0.000000e+00,9.803576e+06,7.462839e+06
max,100.000000,100.000000,65535.000000,65532.000000,17.000000,1.199999e+08,207964.000000,284602.000000,2.866110e+06,6.270000e+08,...,198636.000000,60.000000,1.100000e+08,7.050000e+07,1.100000e+08,1.100000e+08,1.200000e+08,7.420000e+07,1.200000e+08,1.200000e+08


Read test and validation sets

In [ ]:
print('Test set: ')
df_test['Label'].value_counts()

Test set: 


BENIGN                        139135
DoS Hulk                       57531
PortScan                       39701
DDoS                           32006
DoS GoldenEye                   2573
FTP-Patator                     1983
SSH-Patator                     1474
DoS slowloris                   1449
DoS Slowhttptest                1374
Bot                              489
Web Attack � Brute Force         376
Web Attack � XSS                 163
Infiltration                       9
Web Attack � Sql Injection         5
Heartbleed                         3
Name: Label, dtype: int64

In [ ]:
df_test.describe()

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,278270.000000,278270.000000,278270.000000,2.782700e+05,278270.000000,278270.000000,2.782700e+05,2.782700e+05,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,278270.000000,278270.0,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,278270.000000,278270.000000,278270.000000,278270.000000,2.782700e+05,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.0,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,2.782700e+05,278270.000000,2.782700e+05,278270.000000,278270.000000,278270.000000,278270.000000,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05
mean,45364.905750,5840.633640,7.480749,2.018494e+07,13.425781,15.071391,5.018160e+02,2.777075e+04,185.079276,9.858127,42.498337,62.543191,1437.918892,16.702825,472.989076,598.916973,1.319690e+06,9.153090e+04,2.091054e+06,4.928267e+06,1.541155e+07,2.119001e+05,1.981716e+07,4.173283e+06,5.569102e+06,1.522487e+07,1.619357e+06,1.058695e+07,2.418274e+06,2.028897e+06,6.305801e+06,1.360565e+06,0.055008,0.0,3.183946e+02,3.384120e+02,8.541160e+04,6.162424e+03,7.706957,1472.163140,241.332367,466.822599,8.970325e+05,0.058659,0.055008,0.000079,0.423420,0.383936,0.024473,0.0,0.000079,0.515485,266.229446,42.498337,472.989076,13.425781,5.018160e+02,15.071391,2.777075e+04,10570.921080,1731.267470,10.076965,26.450577,9.950258e+04,4.357601e+04,1.635863e+05,7.265777e+04,1.389976e+07,9.310342e+05,1.461183e+07,1.319746e+07
std,17986.051427,15731.041102,3.759347,3.747672e+07,1208.778536,1591.037162,7.898116e+03,3.604963e+06,392.614677,61.235977,99.578231,132.298567,2586.487090,46.983524,797.823718,1144.520735,2.365354e+07,2.939540e+05,5.446991e+06,1.046406e+07,3.156539e+07,3.035605e+06,3.741861e+07,1.178049e+07,1.262896e+07,3.161073e+07,1.066533e+07,2.913323e+07,1.045182e+07,7.731527e+06,2.046630e+07,9.753000e+06,0.227996,0.0,2.524765e+04,3.182466e+04,2.889885e+05,3.511581e+04,17.933308,2584.335069,383.113699,824.111902,2.290032e+06,0.234986,0.227996,0.008891,0.494102,0.486344,0.154512,0.0,0.008891,0.551013,420.649091,99.578231,797.823718,1208.778536,7.898116e+03,1591.037162,3.604963e+06,17760.016938,7434.284715,1153.408495,6.864564,7.423962e+05,4.382250e+05,1.087195e+06,6.664087e+05,3.055265e+07,6.557272e+06,3.158812e+07,3.024680e+07
min,0.000000,0.000000,0.000000,-1.000000e+00,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.200000e+07,-2.000000e+06,-1.000000e+00,0.000000e+00,-1.000000e+00,-1.300000e+

In [ ]:
print('Validation set: ')
df_val['Label'].value_counts()

Validation set: 


BENIGN                        139135
DoS Hulk                       57531
PortScan                       39701
DDoS                           32006
DoS GoldenEye                   2573
FTP-Patator                     1983
SSH-Patator                     1474
DoS slowloris                   1449
DoS Slowhttptest                1374
Bot                              489
Web Attack � Brute Force         376
Web Attack � XSS                 163
Infiltration                       9
Web Attack � Sql Injection         5
Heartbleed                         3
Name: Label, dtype: int64

In [ ]:
df_val.describe()

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,278270.000000,278270.000000,278270.000000,2.782700e+05,278270.000000,278270.000000,278270.000000,2.782700e+05,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,278270.000000,278270.0,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,278270.000000,278270.000000,278270.000000,278270.000000,2.782700e+05,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.0,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,278270.000000,2.782700e+05,278270.000000,278270.000000,278270.000000,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05,2.782700e+05
mean,42404.657796,4584.284134,8.772361,2.267720e+07,5.697168,5.401071,363.328418,6.993901e+03,146.271053,12.216883,37.118951,48.506259,1644.006185,31.044985,545.020851,677.301293,1.026168e+06,7.647836e+04,2.035292e+06,5.660128e+06,1.867983e+07,1.176209e+05,2.242947e+07,4.097971e+06,7.275898e+06,1.857577e+07,8.532117e+05,1.033244e+07,2.108780e+06,2.497470e+06,6.852268e+06,8.777106e+05,0.032285,0.0,-1.853584e+04,-6.184786e+03,6.922916e+04,7.292921e+03,11.819104,1667.675854,265.849753,521.021331,1.032910e+06,0.063622,0.032285,0.000111,0.325008,0.360617,0.052697,0.0,0.000111,0.635189,293.529747,37.118951,545.020851,5.697168,363.328418,5.401071,6.994360e+03,8300.353962,1196.878449,2.850606,-6.302925e+03,1.011663e+05,3.866528e+04,1.584032e+05,7.772919e+04,1.758377e+07,9.337417e+05,1.829755e+07,1.689405e+07
std,18921.321337,13260.493797,4.780477,3.957518e+07,67.142466,87.166823,3246.913595,2.585565e+05,380.776699,29.589643,80.881268,118.838474,2798.340693,63.404307,851.938455,1229.566919,1.848896e+07,2.733855e+05,4.353752e+06,1.081619e+07,3.532609e+07,1.940212e+06,3.955407e+07,9.992215e+06,1.450108e+07,3.536583e+07,7.979563e+06,2.928691e+07,9.212534e+06,9.080364e+06,2.268132e+07,8.098570e+06,0.176757,0.0,4.255039e+06,8.563325e+05,2.681766e+05,3.540818e+04,21.282926,2802.004249,395.033818,872.649085,2.337799e+06,0.244078,0.176757,0.010554,0.468379,0.480181,0.223428,0.0,0.010554,0.543604,431.308583,80.881268,851.938455,67.142466,3246.913595,87.166823,2.586082e+05,13668.065695,6710.083319,32.991321,7.286969e+05,8.404515e+05,4.487356e+05,1.167019e+06,7.582087e+05,3.455426e+07,6.638199e+06,3.540421e+07,3.437331e+07
min,0.000000,0.000000,0.000000,-1.200000e+01,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.200000e+07,-2.000000e+06,-1.200000e+01,0.000000e+00,-1.200000e+01,-1.200000e+01,0.000000e+

## Step 3. Encode Datasets

Encoding the labels, and generate numpy array. Note that the label has not been encoded as one-hot coding. We will use one-hot code later. 

### Step 3.1 Encoding train dataset

In [111]:
df_label = df_train['Label']
data = df_train.drop(columns=['Label'])
Xtrain = data.values
y_train = encode_label(df_label.values)

 ### Step 3.2. SMOTE train data set  

In [112]:
label = encode_label(df_label.values)
bb=label.reshape(label.shape[0],)    
y10 = np.int32(bb)

In [113]:
from collections import Counter

sorted(Counter(y10).items())

[(0, 278274),
 (1, 978),
 (2, 64012),
 (3, 5146),
 (4, 115062),
 (5, 2749),
 (6, 2898),
 (7, 3967),
 (8, 5),
 (9, 18),
 (10, 79402),
 (11, 2948),
 (12, 753),
 (13, 10),
 (14, 326)]

In [114]:
from imblearn.over_sampling import SMOTE
import time
time_start = time.time()

guo = 5000  #Oversampling samples

smo = SMOTE(sampling_strategy={1:guo,5:guo,6:guo,7:guo,8:guo,9:guo,11:guo,12:guo,13:guo,14:guo},random_state=42,k_neighbors=4)

Xtrain_smo, ytrain_smo = smo.fit_sample(Xtrain, y10)   
print(sorted(Counter(ytrain_smo).items()))

time_end = time.time()
time = time_end - time_start
print("time:",time)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

[(0, 278274), (1, 5000), (2, 64012), (3, 5146), (4, 115062), (5, 5000), (6, 5000), (7, 5000), (8, 5000), (9, 5000), (10, 79402), (11, 5000), (12, 5000), (13, 5000), (14, 5000)]
time: 2.867403984069824


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [115]:
sorted(Counter(ytrain_smo).items())

[(0, 278274),
 (1, 5000),
 (2, 64012),
 (3, 5146),
 (4, 115062),
 (5, 5000),
 (6, 5000),
 (7, 5000),
 (8, 5000),
 (9, 5000),
 (10, 79402),
 (11, 5000),
 (12, 5000),
 (13, 5000),
 (14, 5000)]

### Step 3.3. Encoding test dataset

In [116]:
df_label = df_test['Label']
data = df_test.drop(columns=['Label'])
Xtest = data.values
y_test = encode_label(df_label.values)

### Step 3.4 Encoding validation dataset

In [117]:
df_label = df_val['Label']
data = df_val.drop(columns=['Label'])
Xval = data.values
y_val = encode_label(df_label.values)

## Step 4. Normalization or Standardization

The continuous feature values are normalized into the same feature space. This is important when using features that have different measurements, and is a general requirement of many machine learning algorithms. We implement the two methods to see the impact on the final classifications. 

## Option 1. Normalization

The values of the datasets are normalized using the Min-Max scaling technique, bringing them all within a range of [0,1].

### Step 4.1 Normalizing train dataset

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(Xtrain)
X_train

array([[7.66048676e-01, 5.93603125e-03, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.29167620e-01, 1.22077764e-03, 3.52941176e-01, ...,
        0.00000000e+00, 7.12500000e-01, 7.12500000e-01],
       [7.86144808e-01, 6.76005616e-03, 3.52941176e-01, ...,
        8.96864890e-03, 4.90833333e-01, 4.83333333e-01],
       ...,
       [2.53009842e-01, 1.22077764e-03, 3.52941176e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.85610742e-01, 8.08765183e-04, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.26276036e-01, 1.22077764e-03, 3.52941176e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

### Step 4.2. Normalizing validation dataset

In [ ]:
X_val = scaler.fit_transform(Xval)
X_val

array([[4.37994965e-01, 1.22344737e-03, 3.52941176e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.41290913e-01, 6.12488339e-02, 3.52941176e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.87327382e-02, 6.77483980e-03, 3.52941176e-01, ...,
        5.68691207e-05, 8.33333333e-02, 8.32947583e-02],
       ...,
       [9.30510414e-01, 8.10533882e-04, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.50492103e-01, 8.68494701e-02, 3.52941176e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.24353399e-01, 1.22344737e-03, 3.52941176e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

### Step 4.3. Normalizing test dataset

In [ ]:
X_test = scaler.fit_transform(Xtest)
X_test

array([[5.45487838e-01, 9.66383862e-01, 3.52941176e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.17142247e-01, 1.22074038e-03, 3.52941176e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.92001099e-01, 7.97448653e-02, 3.52941176e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [9.04507584e-01, 1.22074038e-03, 3.52941176e-01, ...,
        0.00000000e+00, 8.33370661e-01, 8.33370661e-01],
       [9.22345653e-01, 8.08740501e-04, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.45030061e-01, 6.75984985e-03, 3.52941176e-01, ...,
        5.01804492e-03, 4.88761059e-01, 4.84412289e-01]])

## Option 2. Standardization

In [118]:
from sklearn.preprocessing import StandardScaler

In [119]:
scaler = StandardScaler()

X_train = scaler.fit_transform(Xtrain_smo)
X_val = scaler.fit_transform(Xval)
X_test = scaler.fit_transform(Xtest)

X_train

array([[ 0.93557229,  0.82244604,  0.41557473, ...,  6.21513618,
         1.67674695, -0.20936771],
       [-1.01166095, -1.29871243,  0.76946389, ..., -0.17047892,
        -0.50041048, -0.46384333],
       [ 0.79133279,  0.66532319,  0.82270645, ..., -0.17047892,
        -0.50041048, -0.46384333],
       ...,
       [-1.06186019, -1.45583528,  0.65195724, ..., -0.17047892,
        -0.50041048, -0.46384333],
       [-1.19808882, -1.5130887 ,  0.59009003, ..., -0.17047892,
        -0.50041048, -0.46384333],
       [-1.15878669, -1.60682354,  0.67217655, ..., -0.17047892,
        -0.50041048, -0.46384333]])

## Step 5 One-hot encoding for labels

y_train, y_test and y_val have to be one-hot-encoded. That means they must have dimension (number_of_samples, 15), where 15 denotes number of classes.

In [120]:
from tensorflow.keras.utils import to_categorical

Save the labels for AdaBoostClassifier

In [121]:
y_train_ada = y_train
y_test_ada = y_test
y_val_ada = y_val

In [122]:
y_train = to_categorical(ytrain_smo, 15)
y_test = to_categorical(y_test, 15)
y_val = to_categorical(y_val, 15)

## Step 6. Define the metrics

In [123]:
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier

#importing confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn import metrics
from sklearn.metrics import accuracy_score

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error,mean_absolute_error

Get the attacks' names

In [124]:
labels_d = make_value2index(df_test['Label'])

In [125]:
print(labels_d)

{'BENIGN': 139134, 'Bot': 139623, 'DDoS': 171629, 'DoS GoldenEye': 174202, 'DoS Hulk': 231733, 'DoS Slowhttptest': 233107, 'DoS slowloris': 234556, 'FTP-Patator': 236539, 'Heartbleed': 236542, 'Infiltration': 236551, 'PortScan': 276252, 'SSH-Patator': 277726, 'Web Attack � Brute Force': 278102, 'Web Attack � Sql Injection': 278107, 'Web Attack � XSS': 278270}


# CNN1D Model 

In [126]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, Flatten, Dense, Activation,Dropout,MaxPooling1D
from tensorflow.keras.constraints import max_norm

In [127]:
#hyper-params
batch_size = 7000 # increasing batch size with more gpu added
input_dim = X_train.shape[1]
num_class = 15                   # 15 intrusion classes, including benign traffic class
num_epochs = 30
learning_rates = 1e-4
regularizations = 1e-3
optim = tf.keras.optimizers.Adam(lr=learning_rates, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

print(input_dim)
print(num_class)

73
15


In [128]:
#X_train_r = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train_r = np.zeros((len(X_train), input_dim, 1))
X_train_r[:, :, 0] = X_train[:, :input_dim]
print(X_train_r.shape)

(591896, 73, 1)


In [129]:
X_test_r = np.zeros((len(X_test), input_dim, 1))
X_test_r[:, :, 0] = X_test[:, :input_dim]
print(X_test_r.shape)

(278271, 73, 1)


In [130]:
X_val_r = np.zeros((len(X_val), input_dim, 1))
X_val_r[:, :, 0] = X_val[:, :input_dim]
print(X_val_r.shape)

(278271, 73, 1)


In [138]:
model = Sequential()

# input layer
model.add(Conv1D(filters=32, kernel_size=23, activation='relu', padding='same', kernel_initializer='he_uniform', input_shape=(73,1)))
model.add(Conv1D(filters=32, kernel_size=19, activation='relu', padding='same', kernel_initializer='he_uniform'))
model.add(MaxPooling1D(pool_size=2,strides=2))
model.add(Dropout(0.3))
model.add(BatchNormalization(axis=1))

model.add(Conv1D(filters=64, kernel_size=23, activation='relu', padding='same', kernel_initializer='he_uniform'))
model.add(Conv1D(filters=64, kernel_size=19, activation='relu', padding='same', kernel_initializer='he_uniform'))
model.add(MaxPooling1D(pool_size=2,strides=2))
model.add(Dropout(0.3))
model.add(BatchNormalization(axis=1)) 

model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_class))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_120 (Conv1D)          (None, 73, 32)            768       
_________________________________________________________________
conv1d_121 (Conv1D)          (None, 73, 32)            19488     
_________________________________________________________________
max_pooling1d_60 (MaxPooling (None, 36, 32)            0         
_________________________________________________________________
dropout_80 (Dropout)         (None, 36, 32)            0         
_________________________________________________________________
batch_normalization_60 (Batc (None, 36, 32)            144       
_________________________________________________________________
conv1d_122 (Conv1D)          (None, 36, 64)            47168     
_________________________________________________________________
conv1d_123 (Conv1D)          (None, 36, 64)          

In [139]:
from keras.optimizers import Nadam, Adam, RMSprop, SGD
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
import keras
import time
time_start = time.time()

reduce_lr = keras.callbacks.ReduceLROnPlateau(moniter='val_loss',
                                              factor=0.1,
                                              patience=10)
#optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.004)
# optimizer =  Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
optimizer = Nadam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.0001)
model.compile(loss = "categorical_crossentropy",optimizer = optimizer, metrics = ["accuracy"])

history = model.fit(X_train_r, y_train, 
                    epochs=10, 
                    batch_size=batch_size, 
                    verbose=2,
                    validation_data=(X_val_r, y_val),
                    callbacks=[reduce_lr])
time_end = time.time()
train_time = time_end - time_start
print("train_time:",train_time)

Epoch 1/10
85/85 - 15s - loss: 0.5806 - accuracy: 0.8390 - val_loss: 0.2371 - val_accuracy: 0.9737
Epoch 2/10
85/85 - 14s - loss: 0.1020 - accuracy: 0.9678 - val_loss: 0.0937 - val_accuracy: 0.9762
Epoch 3/10
85/85 - 14s - loss: 0.0635 - accuracy: 0.9795 - val_loss: 0.0591 - val_accuracy: 0.9749
Epoch 4/10
85/85 - 13s - loss: 0.0459 - accuracy: 0.9849 - val_loss: 0.0424 - val_accuracy: 0.9874
Epoch 5/10
85/85 - 13s - loss: 0.0383 - accuracy: 0.9875 - val_loss: 0.0410 - val_accuracy: 0.9884
Epoch 6/10
85/85 - 13s - loss: 0.0328 - accuracy: 0.9890 - val_loss: 0.0494 - val_accuracy: 0.9867
Epoch 7/10
85/85 - 13s - loss: 0.3878 - accuracy: 0.9078 - val_loss: 0.0625 - val_accuracy: 0.9744
Epoch 8/10
85/85 - 13s - loss: 0.0448 - accuracy: 0.9855 - val_loss: 0.0535 - val_accuracy: 0.9821
Epoch 9/10
85/85 - 13s - loss: 0.0340 - accuracy: 0.9888 - val_loss: 0.0529 - val_accuracy: 0.9834
Epoch 10/10
85/85 - 13s - loss: 0.0295 - accuracy: 0.9900 - val_loss: 0.0491 - val_accuracy: 0.9857
train_tim

## Get the metrics

In [140]:
# evaluate model
accuracy = model.evaluate(X_test_r, y_test, batch_size=batch_size, verbose=1)

40/40 [==============================] - 2s 37ms/step - loss: 0.1467 - accuracy: 0.9751


In [141]:
y_pred=model.predict(X_test_r)

In [142]:
display_metrics(y_test_ada, np.argmax(y_pred, axis = 1), labels_d)


Accuracy: 0.98

Micro Precision: 0.98
Micro Recall: 0.98
Micro F1-score: 0.98

Macro Precision: 0.61
Macro Recall: 0.65
Macro F1-score: 0.60

Weighted Precision: 0.98
Weighted Recall: 0.98
Weighted F1-score: 0.97

Classification Report

                            precision    recall  f1-score   support

                    BENIGN       0.99      0.99      0.99    139135
                       Bot       0.00      0.00      0.00       489
                      DDoS       0.94      1.00      0.97     32006
             DoS GoldenEye       0.97      0.84      0.90      2573
                  DoS Hulk       0.99      0.96      0.98     57531
          DoS Slowhttptest       0.61      0.77      0.68      1374
             DoS slowloris       0.74      0.50      0.59      1449
               FTP-Patator       0.95      0.97      0.96      1983
                Heartbleed       0.50      1.00      0.67         3
              Infiltration       0.00      0.00      0.00         9
             